<a href="https://colab.research.google.com/github/GalSerge/methods_ml_2/blob/main/ex4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ниже приведен код со ссылкой на набор данных «happy» или «sad», содержащий 80 изображений, 40 счастливых и 40 грустных.
Создайте сверточную нейронную сеть, которая обучается со 100% точностью на этих изображениях, с отменой обучения при достижении точности обучения .999

In [10]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import zipfile
#
!wget --no-check-certificate "https://github.com/andrew-veriga/Tensorflow-labs/raw/master/happy-or-sad.zip" \
   -O "/tmp/happy-or-sad.zip"


--2024-04-12 12:18:25--  https://github.com/andrew-veriga/Tensorflow-labs/raw/master/happy-or-sad.zip
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/andrew-veriga/Tensorflow-labs/master/happy-or-sad.zip [following]
--2024-04-12 12:18:26--  https://raw.githubusercontent.com/andrew-veriga/Tensorflow-labs/master/happy-or-sad.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  --.-KB/s    in 0.01s   

2024-04-12 12:18:26 (176 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/267

In [2]:

zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

import os

train_happy_dir = os.path.join('/tmp/h-or-s/happy')

train_sad_dir = os.path.join('/tmp/h-or-s/sad')

train_happy_names = os.listdir(train_happy_dir)
print(train_happy_names[:10])

train_sad_names = os.listdir(train_sad_dir)
print(train_sad_names[:10])

['happy1-17.png', 'happy2-13.png', 'happy2-16.png', 'happy2-07.png', 'happy1-18.png', 'happy2-17.png', 'happy2-01.png', 'happy1-06.png', 'happy1-11.png', 'happy2-02.png']
['sad2-01.png', 'sad2-15.png', 'sad2-11.png', 'sad1-08.png', 'sad1-05.png', 'sad1-19.png', 'sad2-08.png', 'sad1-04.png', 'sad2-00.png', 'sad2-02.png']


In [3]:
import os

train_happy_dir = os.path.join('/tmp/h-or-s/happy')

train_sad_dir = os.path.join('/tmp/h-or-s/sad')

train_happy_names = os.listdir(train_happy_dir)
print(train_happy_names[:10])

train_sad_names = os.listdir(train_sad_dir)
print(train_sad_names[:10])

['happy1-17.png', 'happy2-13.png', 'happy2-16.png', 'happy2-07.png', 'happy1-18.png', 'happy2-17.png', 'happy2-01.png', 'happy1-06.png', 'happy1-11.png', 'happy2-02.png']
['sad2-01.png', 'sad2-15.png', 'sad2-11.png', 'sad1-08.png', 'sad1-05.png', 'sad1-19.png', 'sad2-08.png', 'sad1-04.png', 'sad2-00.png', 'sad2-02.png']


In [6]:
class StopOnPoint(tf.keras.callbacks.Callback):
    def __init__(self, point):
        super(StopOnPoint, self).__init__()
        self.point = point

    def on_epoch_end(self, epoch, logs=None):
        accuracy = logs['accuracy']
        if accuracy >= self.point:
            self.model.stop_training = True

In [11]:
DESIRED_ACCURACY = 0.999

callbacks = StopOnPoint(DESIRED_ACCURACY)

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    '/tmp/h-or-s',
    target_size=(150, 150),
    batch_size=80,
    class_mode='binary'
)

model.fit(train_generator, epochs=10, callbacks=[callbacks])
print('Достигнута точность 99.9%, поэтому обучение закончено!')

loss, accuracy = model.evaluate(train_generator)
print(f'Точность на обучающих данных: {accuracy*100:.2f}%')


Found 80 images belonging to 2 classes.
Epoch 1/10
1/1 [==============================] - 5s 5s/step - loss: 0.6892 - accuracy: 0.5000
Epoch 2/10
1/1 [==============================] - 3s 3s/step - loss: 1.6884 - accuracy: 0.5000
Epoch 3/10
1/1 [==============================] - 4s 4s/step - loss: 0.6852 - accuracy: 0.5000
Epoch 4/10
1/1 [==============================] - 3s 3s/step - loss: 0.6925 - accuracy: 0.5000
Epoch 5/10
1/1 [==============================] - 3s 3s/step - loss: 0.6805 - accuracy: 0.5000
Epoch 6/10
1/1 [==============================] - 3s 3s/step - loss: 0.6685 - accuracy: 0.9250
Epoch 7/10
1/1 [==============================] - 8s 8s/step - loss: 0.6564 - accuracy: 0.7500
Epoch 8/10
1/1 [==============================] - 4s 4s/step - loss: 0.6403 - accuracy: 1.0000
Достигнута точность 99.9%, поэтому обучение закончено!
1/1 [==============================] - 1s 1s/step - loss: 0.6149 - accuracy: 1.0000
Точность на обучающих данных: 100.00%
